In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Columbia EECS E6893 Big Data Analytics
"""
This module is the spark streaming analysis process.


Usage:
    If used with dataproc:
        gcloud dataproc jobs submit pyspark --cluster <Cluster Name> twitterHTTPClient.py

    Create a dataset in BigQurey first using
        bq mk bigdata_sparkStreaming

    Remeber to replace the bucket with your own bucket name


Todo:
    1. hashtagCount: calculate accumulated hashtags count
    2. wordCount: calculate word count every 60 seconds
        the word you should track is listed below.
    3. save the result to google BigQuery

"""

from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
import time
import subprocess
import re
from google.cloud import bigquery

# global variables
bucket = " "    # TODO : replace with your own bucket name
output_directory_hashtags = 'gs://{}/hadoop/tmp/bigquery/pyspark_output/hashtagsCount'.format(bucket)
output_directory_wordcount = 'gs://{}/hadoop/tmp/bigquery/pyspark_output/wordcount'.format(bucket)

# output table and columns name
output_dataset = ' '                     #the name of your dataset in BigQuery
output_table_hashtags = 'hashtags'
columns_name_hashtags = ['hashtags', 'count']
output_table_wordcount = 'wordcount'
columns_name_wordcount = ['word', 'count', 'time']

# parameter
IP = 'localhost'    # ip port
PORT = 9001       # port

STREAMTIME = 600          # time that the streaming process runs 600

WORD = ['data', 'spark', 'ai', 'movie', 'good']     #the words you should filter and do word count

# Helper functions
def saveToStorage(rdd, output_directory, columns_name, mode):
    """
    Save each RDD in this DStream to google storage
    Args:
        rdd: input rdd
        output_directory: output directory in google storage
        columns_name: columns name of dataframe
        mode: mode = "overwirte", overwirte the file
              mode = "append", append data to the end of file
    """
    if not rdd.isEmpty():
        (rdd.toDF( columns_name ) \
        .write.save(output_directory, format="json", mode=mode))


def saveToBigQuery(sc, output_dataset, output_table, directory):
    """
    Put temp streaming json files in google storage to google BigQuery
    and clean the output files in google storage
    """
    files = directory + '/part-*'
    subprocess.check_call(
        'bq load --source_format NEWLINE_DELIMITED_JSON '
        '--replace '
        '--autodetect '
        '{dataset}.{table} {files}'.format(
            dataset=output_dataset, table=output_table, files=files
        ).split())
    output_path = sc._jvm.org.apache.hadoop.fs.Path(directory)
    output_path.getFileSystem(sc._jsc.hadoopConfiguration()).delete(
        output_path, True)


def hashtagCount(words):
    """
    Calculate the accumulated hashtags count sum from the beginning of the stream
    and sort it by descending order of the count.
    Ignore case sensitivity when counting the hashtags:
        "#Ab" and "#ab" is considered to be a same hashtag
    You have to:
    1. Filter out the word that is hashtags.
       Hashtag usually start with "#" and followed by a serious of alphanumeric
    2. map (hashtag) to (hashtag, 1)
    3. sum the count of current DStream state and previous state
    4. transform unordered DStream to a ordered Dstream
    Hints:
        you may use regular expression to filter the words
        You can take a look at updateStateByKey and transform transformations
    Args:
        dstream(DStream): stream of real time tweets
    Returns:
        DStream Object with inner structure (hashtag, count)
    """

    # TODO: insert your code here
    def updateFunc(new_values, last_sum):
        return sum(new_values) + (last_sum or 0)

    hashtag = words.map(lambda x: x.lower()).filter(
        lambda x: len(x) > 2 and x[0] == "#").map(
        lambda x: (x, 1))
    hashtag_cnt = hashtag.reduceByKey(lambda cnt1, cnt2: cnt1 + cnt2)
    hashtag_cnt_total = hashtag_cnt.updateStateByKey(updateFunc).transform(
        lambda rdd: rdd.sortBy(lambda x: x[1], ascending=False))
    return hashtag_cnt_total
    pass

def wordCount(words):
    """
    Calculte the count of 5 sepcial words for every 60 seconds (window no overlap)
    You can choose your own words.
    Your should:
    1. filter the words
    2. count the word during a special window size
    3. add a time related mark to the output of each window, ex: a datetime type
    Hints:
        You can take a look at reduceByKeyAndWindow transformation
        Dstream is a serious of rdd, each RDD in a DStream contains data from a certain interval
        You may want to take a look of transform transformation of DStream when trying to add a time
    Args:
        dstream(DStream): stream of real time tweets
    Returns:
        DStream Object with inner structure (word, (count, time))
    """

    # TODO: insert your code here
    word_cnt = words.map(lambda x: x.lower()).filter(lambda x: x in WORD).map(
        lambda x: (x, 1)).reduceByKeyAndWindow(lambda x, y: x + y,
                                               lambda x, y: x - y, 60, 60)
    word_cnt_total = word_cnt.transform(
        lambda time, rdd: rdd.map(
            lambda x: (x[0], x[1], time.strftime("%Y-%m-%d %H:%M:%S"))))
    return word_cnt_total
    pass


if __name__ == '__main__':
    # Spark settings
    conf = SparkConf()
    conf.setMaster('local[2]')
    conf.setAppName("TwitterStreamApp")

    # create spark context with the above configuration
    sc = SparkContext(conf=conf)
    sc.setLogLevel("ERROR")

    # create sql context, used for saving rdd
    sql_context = SQLContext(sc)

    # create the Streaming Context from the above spark context with batch interval size 5 seconds
    ssc = StreamingContext(sc, 5)
    # setting a checkpoint to allow RDD recovery
    ssc.checkpoint("~/checkpoint_TwitterApp")

    # read data from port 9001
    dataStream = ssc.socketTextStream(IP, PORT)
    dataStream.pprint()

    words = dataStream.flatMap(lambda line: line.split(" "))

    # calculate the accumulated hashtags count sum from the beginning of the stream
    topTags = hashtagCount(words)
    topTags.pprint()

    # Calculte the word count during each time period 6s
    wordCount = wordCount(words)
    wordCount.pprint()

    # save hashtags count and word count to google storage
    # used to save to google BigQuery
    # You should:
    #   1. topTags: only save the lastest rdd in DStream
    #   2. wordCount: save each rdd in DStream
    # Hints:
    #   1. You can take a look at foreachRDD transformation
    #   2. You may want to use helper function saveToStorage
    #   3. You should use save output to output_directory_hashtags, output_directory_wordcount,
    #       and have output columns name columns_name_hashtags and columns_name_wordcount.
    # TODO: insert your code here
    topTags.foreachRDD(lambda rdd: saveToStorage(rdd, output_directory_hashtags,
                                                 columns_name_hashtags,
                                                 mode="overwrite"))
    wordCount.foreachRDD(
        lambda rdd: saveToStorage(rdd, output_directory_wordcount,
                                  columns_name_wordcount, mode="append"))
    # start streaming process, wait for 600s and then stop.
    ssc.start()
    time.sleep(STREAMTIME)
    ssc.stop(stopSparkContext=False, stopGraceFully=True)

    # put the temp result in google storage to google BigQuery
    saveToBigQuery(sc, output_dataset, output_table_hashtags, output_directory_hashtags)
    saveToBigQuery(sc, output_dataset, output_table_wordcount, output_directory_wordcount)


-------------------------------------------
Time: 2022-10-23 04:56:35
-------------------------------------------

-------------------------------------------
Time: 2022-10-23 04:56:35
-------------------------------------------

-------------------------------------------
Time: 2022-10-23 04:56:40
-------------------------------------------
モイ！iPhoneから #ツイキャスゲームズ を開始しました プレイルームをプレイ #ツイキャスゲームズ
https://t.co/8cztSGloojモイ！iPhoneから #ツイキャスゲームズ を開始しました プレイルームをプレイ #ツイキャスゲームズ
https://t.co/8cztSGloojRT @WandasAttorney: Black Adam should’ve been a Justice Society movie ft. Black Adam not the other way around… https://t.co/MQ90Dv0LztRT @WandasAttorney: Black Adam should’ve been a Justice Society movie ft. Black Adam not the other way around… https://t.co/MQ90Dv0LztRT @T_Az38: AIを操る者が新たに生み出してしまった「爆乳機関車」とかいう業の深いジャンル、このために生み出された造語かと思いきや、元々特定の形式の機関車を指す言葉として存在してた上に、その言葉の由来がハッキリ分かってないの面白すぎる（RT @T_Az38: AIを操る者が新たに生み出してしまった「爆乳機関車」とかいう業の深いジャンル、このために生み出された造語かと思いきや、元々特定の形式の機関車を指す言葉として存在してた上に、その言葉の由来がハッキリ分かっ

-------------------------------------------
Time: 2022-10-23 04:57:05
-------------------------------------------
と＿＿言ぃながら買ってる人ㅋㅋ https://t.co/zYcXzZCa2LRT @kotarotatsumi: YouTubeはこちら
https://t.co/LdkycDMgSL
https://t.co/tytHbAHhy6RT @kotarotatsumi: YouTubeはこちら
https://t.co/LdkycDMgSL
https://t.co/tytHbAHhy6@loviebiblee aaaa makasiii, iya lucu 😭😭😭🙏🏻@loviebiblee aaaa makasiii, iya lucu 😭😭😭🙏🏻RT @euqIute: odeio aquela etapa da TPM em que vc tem NOÇÃO de que ta insuportável mas não consegue parar de ser chata ai entra em desespero…RT @euqIute: odeio aquela etapa da TPM em que vc tem NOÇÃO de que ta insuportável mas não consegue parar de ser chata ai entra em desespero…RT @APrime02: (Pict &amp; Video)
Fileupload:
https://t.co/GbrC33SMEs
https://t.co/wnsGLXRsFY https://t.co/9vQUiAIEmpRT @APrime02: (Pict &amp; Video)
Fileupload:
https://t.co/GbrC33SMEs
...

-------------------------------------------
Time: 2022-10-23 04:57:05
-------------------------------------------
('#onpassive', 4)
('#nf

-------------------------------------------
Time: 2022-10-23 04:57:30
-------------------------------------------
https://t.co/W3HNYvJhLdThe most random movies be free on YouTube like Scary Movie 4The most random movies be free on YouTube like Scary Movie 4@AlimShan_ Means it was bad movie@AlimShan_ Means it was bad movieIMFRGIGNG WHY WAS THAT ACTUALLY SO SCARYYYYFJJ I HAVWNT BEEN THAT SCARED OF A MOVIE IN A MINUTE I WAS CLUTCHING MY PEARLS BY THE END OF ITIMFRGIGNG WHY WAS THAT ACTUALLY SO SCARYYYYFJJ I HAVWNT BEEN THAT SCARED OF A MOVIE IN A MINUTE I WAS CLUTCHING MY PEARLS BY THE END OF IT@liz_churchill7 Watch this with the sound off. Looks like a maniacal character out of a Batman movie.@liz_churchill7 Watch this with the sound off. Looks like a maniacal character out of a Batman movie.@47_babeth @France61224153 Mdr pareil tabou ma babeth@47_babeth @France61224153 Mdr pareil tabou ma babethRT @nknmttb_nitijo: あの…某スレで『爆乳機関車』って見たんですけど……AIクン壊れちゃう…… https://t.co/fApH5WKVqPRT @nknmttb_n

-------------------------------------------
Time: 2022-10-23 04:57:45
-------------------------------------------
Linkfull https://t.co/L8kAY…@RespectForMy Crtz q a de 2002 ainda tá nascendo (eu sei matemática gente foi só piada@RespectForMy Crtz q a de 2002 ainda tá nascendo (eu sei matemática gente foi só piadaenjoying the first segment so farenjoying the first segment so far@Shields10Curtis @Aubxbrx Hey handsome how are you doing https://t.co/2bzFqVnKeM@Shields10Curtis @Aubxbrx Hey handsome how are you doing https://t.co/2bzFqVnKeMbuti spark collector ako https://t.co/xomz0ogFWpbuti spark collector ako https://t.co/xomz0ogFWp@Elvispatrick20 @Harmless12345 There are many ways for politicians to scam people. Risking your life on a canoe in the trenches and flooded marshes with disease and reptiles as your companions is not one O them, except you are acting a movie.@Elvispatrick20 @Harmless12345 There are many ways for politicians to scam people. Risking your life on a canoe in the tre

-------------------------------------------
Time: 2022-10-23 04:58:10
-------------------------------------------
#Jailer #Rajinikanth https://t.co/7MJik54MXW@Stassie2003 Nah gang yall content falling off 💀@Stassie2003 Nah gang yall content falling off 💀RT @Tsukuyomi_Ai: ベストポジション発見😎ここにすみます🏠 #メンテの日じゃないけどss貼る https://t.co/K5CRaHI4TQRT @Tsukuyomi_Ai: ベストポジション発見😎ここにすみます🏠 #メンテの日じゃないけどss貼る https://t.co/K5CRaHI4TQ@daisymay4263 Even the article repeats the environmental claims after clarifying it was only ever one issue, over the rayysism claims.@daisymay4263 Even the article repeats the environmental claims after clarifying it was only ever one issue, over the rayysism claims.RT @WarpsiwaMovie: Kaname Momojiri
Full Movie https://t.co/ZMbOQja2Es

👑SA-WINNER👑
💵สมัครใหม่โบนัส 100% สูงสุด 2,000
💵คืนยอดเสีย 20% ทุกวัน…RT @WarpsiwaMovie: Kaname Momojiri
Full Movie https://t.co/ZMbOQja2Es

👑SA-WINNER👑
💵สมัครใหม่โบนัส 100% สูงสุด 2,000
...

-------------------------------------------
Time: 2022-10-23

...

-------------------------------------------
Time: 2022-10-23 05:04:45
-------------------------------------------
https://t.co/xKD97XSo1s@JadeLfbv_ Oui comme moi j'ai dit à mon ex que je l'aimais alors que j'étais victime. Elle précise que elle était fatiguée en fait, ça suffit à tout comprendre@JadeLfbv_ Oui comme moi j'ai dit à mon ex que je l'aimais alors que j'étais victime. Elle précise que elle était fatiguée en fait, ça suffit à tout comprendre@CoyJandreau https://t.co/Cjqv6kQRxd@CoyJandreau https://t.co/Cjqv6kQRxdRT @anticor_org: "J’ai donné 600 € et en 2 mois j’ai eu mon HLM". Anticor a déposé plainte avec constitution de partie civile pour corrupti…RT @anticor_org: "J’ai donné 600 € et en 2 mois j’ai eu mon HLM". Anticor a déposé plainte avec constitution de partie civile pour corrupti…because he’s my rock! https://t.co/VVU1vnHipLbecause he’s my rock! https://t.co/VVU1vnHipLRT @Julien__Tardy: Il y a une chose que je me saisis pas dans le drame de l’affaire du meurtre abject d

-------------------------------------------
Time: 2022-10-23 05:05:10
-------------------------------------------
" O cão chupando manga" só que ai serve para os 3 https://t.co/0ZOLfmQtAqRT @pattbb8: Phone out, filming while the movie is still playing, acting like he is on an amusement park ride… Scorsese is once again rightRT @pattbb8: Phone out, filming while the movie is still playing, acting like he is on an amusement park ride… Scorsese is once again rightチーム長野の魅力をAIに伝えるのは難しい。 https://t.co/NIIXcFGpGuチーム長野の魅力をAIに伝えるのは難しい。 https://t.co/NIIXcFGpGuそんなのりで初の
婚活パーティーとやらにいってきた。
AI使用してのマッチングとかで
めちゃくちゃ面白かった。

どストレートに失礼な人から
かっこいいイケメン、
落ち着いた３０代と
いろんな人とお話しできた。

...

-------------------------------------------
Time: 2022-10-23 05:05:10
-------------------------------------------
('#ちょこらび', 1084)
('#ai', 232)
('#nft', 94)
('#aiart', 66)
('#kantara', 64)
('#cloud', 62)
('#prabhas', 54)
('#devops', 50)
('#hiring', 50)
('#競馬ai', 48)
...

-------------------------------------------
Time: 2022-10-23 

-------------------------------------------
Time: 2022-10-23 05:05:40
-------------------------------------------
東京3R◎8.2倍2着◎▲△…Adam Sandler is a true goat BUT Water Boy is elite😂😂💯💯 he got like 5/6 movies tied for 2nd place https://t.co/VGqb099yP4Adam Sandler is a true goat BUT Water Boy is elite😂😂💯💯 he got like 5/6 movies tied for 2nd place https://t.co/VGqb099yP4なんてこった、Aiどこに向かってるんだと嘆けばいいのか

Aiのおかげで形にし難い二ッチな界隈に潤いがと解釈すればいいのか……

わからないけど、知らない何かが力を得ている感じ(;'∀')なんてこった、Aiどこに向かってるんだと嘆けばいいのか

Aiのおかげで形にし難い二ッチな界隈に潤いがと解釈すればいいのか……

わからないけど、知らない何かが力を得ている感じ(;'∀')Jsp j’ai quoi mais j’me suis réveillé énervéJsp j’ai quoi mais j’me suis réveillé énervé@Argent_AI blahaj is so cute i love him@Argent_AI blahaj is so cute i love himRT @_Muzzu_: Diwali is  incomplete without a Thalapathy @actorvijay movie🙁 https://t.co/hgzyPBy0vWRT @_Muzzu_: Diwali is  incomplete without a Thalapathy @actorvijay movie🙁 https://t.co/hgzyPBy0vWわこかに！わこまい！わこさら！
#ちょこらび
...

-------------------------------------------
Time: 202

-------------------------------------------
Time: 2022-10-23 05:06:05
-------------------------------------------
https://t.co/q8wPFCqzTF@DudermanBoyo The best trilogy ever it will never be toped it had a great blend of man made props and miniatures and a wonderful use of CGI and the gorgeous locations I want to visit New Zealand one day@DudermanBoyo The best trilogy ever it will never be toped it had a great blend of man made props and miniatures and a wonderful use of CGI and the gorgeous locations I want to visit New Zealand one daymovie where Jim Carrey dies IRLmovie where Jim Carrey dies IRLRT @realmeninav: 13:40 - amiga ta ai?

23:00 - oi to sim!RT @realmeninav: 13:40 - amiga ta ai?

23:00 - oi to sim!RT @_kinotake_: 凸待ちぃ！！！！！！！！！！
#ちょこらび
https://t.co/cGgHhT8fV6RT @_kinotake_: 凸待ちぃ！！！！！！！！！！
#ちょこらび
https://t.co/cGgHhT8fV6@NintenbronyThe Since when has capitalism been about fairness?

...

-------------------------------------------
Time: 2022-10-23 05:06:05
----------------------

-------------------------------------------
Time: 2022-10-23 05:06:30
-------------------------------------------
('movie', 316, '2022-10-23 05:06:30')
('ai', 196, '2022-10-23 05:06:30')
('good', 36, '2022-10-23 05:06:30')
('spark', 18, '2022-10-23 05:06:30')
('data', 2, '2022-10-23 05:06:30')

-------------------------------------------
Time: 2022-10-23 05:06:35
-------------------------------------------

-------------------------------------------
Time: 2022-10-23 05:06:35
-------------------------------------------
('#ちょこらび', 1152)
('#ai', 258)
('#nft', 102)
('#prabhas', 76)
('#cloud', 74)
('#kantara', 70)
('#aiart', 68)
('#競馬ai', 64)
('#devops', 56)
('#hiring', 56)
...

-------------------------------------------
Time: 2022-10-23 05:06:40
-------------------------------------------

-------------------------------------------
Time: 2022-10-23 05:06:40
-------------------------------------------
('#ちょこらび', 1152)
('#ai', 258)
('#nft', 102)
('#prabhas', 76)
('#cloud', 74)
('#kantara'